<a href="https://colab.research.google.com/github/efredin1/personal_repo/blob/master/new_deliv_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_3_SEC_FILE = '/content/drive/MyDrive/nn-deliv-3/features_3_sec.csv'
DATA_30_SEC_FILE = '/content/drive/MyDrive/nn-deliv-3/features_30_sec.csv'
df = pd.read_csv(DATA_30_SEC_FILE)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [ ]:
df.shape

(1000, 60)

In [ ]:
df = df.drop(labels='filename', axis=1)

In [ ]:
labels = df.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(labels)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(np.array(df.iloc[:, :-1], dtype = float))

In [ ]:
x_train_eval, x_unknown, y_train_eval, y_unknown = train_test_split(x, y, stratify=y, test_size=0.2)
x_train, x_eval, y_train, y_eval = train_test_split(x_train_eval, y_train_eval, stratify=y_train_eval, test_size=0.25)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_eval.shape, y_eval.shape)
print(x_unknown.shape, y_unknown.shape)

(600, 58) (600,)
(200, 58) (200,)
(200, 58) (200,)


In [ ]:
class MusicData(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = x_data
    self.y_data = y_data
        
  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
        
  def __len__ (self):
    return len(self.x_data)

In [ ]:
class MusicMLP(nn.Module):
  def __init__(self, features, classes):
    super(MusicMLP, self).__init__()
    self.input = nn.Linear(features, 256)
    self.hidden1 = nn.Linear(256, 128)
    self.hidden2 = nn.Linear(128, 64)
    self.output = nn.Linear(64, classes)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.relu(self.input(x))
    x = self.relu(self.hidden1(x))
    x = self.relu(self.hidden2(x))
    x = self.softmax(self.output(x))
    return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

train_dataset   = MusicData(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long())
test_dataset    = MusicData(torch.from_numpy(x_eval).float(), torch.from_numpy(y_eval).long())
unknown_dataset = MusicData(torch.from_numpy(x_unknown).float(), torch.from_numpy(y_unknown).long())

cpu


In [ ]:
BATCH_SIZE = 128
SM_BATCH_SIZE = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_dataset, batch_size=SM_BATCH_SIZE)
unknown_loader = DataLoader(dataset=unknown_dataset, batch_size=SM_BATCH_SIZE)

In [ ]:
NUM_FEATURES = x_train.shape[1]
NUM_CLASSES = 10
model = MusicMLP(NUM_FEATURES, NUM_CLASSES)
model.to(device)

MusicMLP(
  (input): Linear(in_features=58, out_features=256, bias=True)
  (hidden1): Linear(in_features=256, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [ ]:
LEARNING_RATE = 0.001
#train_weights = torch.tensor([1./60 for i in range(10)])
criterion = nn.CrossEntropyLoss()
#criterion = nn.CrossEntropyLoss(weight=train_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def evaluate(model, loader, device):
  model.eval()
  total_correct = 0
  for x_actual, y_actual in loader:
    x_actual, y_actual = x_actual.to(device), y_actual.to(device)

    y_pred = model(x_actual)
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, predictions = torch.max(y_pred_softmax, dim=1)
    result_list = (y_actual == predictions).float()
    total_correct += sum(result_list)
  acc = total_correct/len(loader)
  return acc

In [ ]:
MODE = 'eval'
EPOCHS = 50
SAVE_PATH = 'best_model_deliv_3.pth'
train_accs = []
train_losses = []

if MODE == 'train':
  model.train()
  high_acc   = 0.0
  for epoch in range(EPOCHS):
    epoch_loss = 0.0
    epoch_acc  = 0.0

    # Training
    for x_batch, y_batch in train_loader:
      optimizer.zero_grad()

      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)

      y_pred = model(x_batch)

      loss = criterion(y_pred, y_batch)

      loss.backward()
      optimizer.step()

      acc = evaluate(model, test_loader, device).item()

      if acc > high_acc:
        torch.save(model.state_dict(), SAVE_PATH)

    if (epoch + 1) % 5 == 0:
      print(f"Epoch #{epoch+1} Training Accuracy: {acc:.2f}%, Loss: {loss}")

elif MODE == 'eval':
  model.load_state_dict(torch.load(SAVE_PATH))
  acc = evaluate(model, test_loader, device)
  print(f"Test accuracy = {acc}")


Test accuracy = 21.285715103149414
